In [2]:
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)

[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret():

    secret_name = "DBCreds"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    # Parse the secret string to get the credentials
    secret_dict = json.loads(secret)
    username = secret_dict['username']
    password = secret_dict['password']
    host = secret_dict['host']
    port = secret_dict['port']
    dbname = secret_dict['dbname']

    return username, password, host, port, dbname


(user,pswd,host,port,db) = get_secret()

In [2]:
import logging
from kafka import KafkaConsumer
import requests
import json
import psycopg2
import uuid

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

resp = []

# Replace with your Kafka broker address(es)
brokers = [f"{host}:9092"]

# Replace with your topic name
topic = "three"


# KServe inference service URL
kserve_url = 'http://lrc-version4.kubeflow-user-example-com.svc.cluster.local/v1/models/lrc-version4:predict'

# Create a Kafka consumer
logger.info("Creating Kafka consumer...")
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=brokers,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-group',
    value_deserializer=lambda v: v.decode('utf-8')
)

logger.info(f"Subscribed to topic: {topic}")


# Function to send data to KServe
def send_to_kserve(data):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(kserve_url, headers=headers, data=json.dumps(data))
    return response.json()

def outcome_to_database(values):
    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
    # Data to insert
    outcome_data = {
        'uid': values[0],  # Generating a unique ID
        'outcome': values[1]  # This can be 0 or 1
    }

# Connect to PostgreSQL
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        exit()

    # Insert data into the table
    try:
        insert_query = """
        INSERT INTO malware_outcomes (uid, outcome)
        VALUES (%s, %s)
        """
        cursor.execute(insert_query, (outcome_data['uid'], outcome_data['outcome']))
        conn.commit()
        print("Data inserted successfully.")
    except Exception as e:
        print(f"Failed to insert data: {e}")
        conn.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("PostgreSQL connection closed.")

        
        

try:
    for message in consumer:
        try:
            resp = []
            message = json.loads(message.value)  # Message value is already a dict
            print(f"Received message: {message}")
            uid = message.pop('uid', None)
            resp.append(uid)
            #print(f"new message: {message}")
            # Prepare data for KServe
            kserve_payload = {
                "instances": [list(message.values())]  # Reshape the data to 2D array
            }

            # Send data to KServe
            kserve_response = send_to_kserve(kserve_payload)
            resp.append(kserve_response['predictions'][0])
            print(f"KServe response: {kserve_response}")
            print(resp)
            # outcome_to_database(resp)
        except json.JSONDecodeError:
            print(f"Received non-JSON message: {message.value}")
        except Exception as e:
            logger.error(f"Error sending to KServe: {e}")
        #logger.info(f"Partition: {message.partition}, Offset: {message.offset}")
except KeyboardInterrupt:
    logger.info("Consumer stopped manually.")
finally:
    consumer.close()
    logger.info("Consumer closed.")


INFO:__main__:Creating Kafka consumer...
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=54.224.168.37:9092 <connecting> [IPv4 ('54.224.168.37', 9092)]>: connecting to 54.224.168.37:9092 [('54.224.168.37', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=54.224.168.37:9092 <connecting> [IPv4 ('54.224.168.37', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('three',)
INFO:__main__:Subscribed to topic: three
INFO:kafka.conn:<BrokerConnection node_id=2 host=ec2-54-224-168-37.compute-1.amazonaws.com:9094 <connecting> [IPv4 ('54.224.168.37', 9094)]>: connecting to ec2-54-224-168-37.compute-1.amazonaws.com:9094 [('54.224.168.37', 9094) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=2 host=ec2-54-224-

Received message: {'SizeOfOptionalHeader': -0.3605074997311053, 'MinorLinkerVersion': -0.37628545401307756, 'SizeOfCode': 0.7319818837332333, 'SizeOfInitializedData': -0.21890447616437766, 'SizeOfUninitializedData': -0.1275414123374885, 'AddressOfEntryPoint': -0.53416668795529, 'BaseOfCode': -0.1081578679974226, 'ImageBase': -0.014265954069590235, 'SectionAlignment': -0.027858834545492168, 'MajorOperatingSystemVersion': -3.78330104335856, 'MinorOperatingSystemVersion': -0.8158417947209594, 'MinorImageVersion': -0.08601248526283754, 'MajorSubsystemVersion': -8.146929399364778, 'SizeOfHeaders': -0.19993743506090722, 'CheckSum': -0.30014776191072257, 'Subsystem': 18.95581955061661, 'DllCharacteristics': -1.1838642376691677, 'SizeOfStackReserve': 0.3632090004457607, 'SizeOfStackCommit': -0.18911824926073148, 'SizeOfHeapReserve': 0.12754896101368254, 'SectionsNb': 1.6857085473286477, 'SectionsMeanEntropy': 0.9690165176474705, 'SectionsMeanRawsize': -0.08980875755177545, 'SectionsMeanVirtual

INFO:__main__:Consumer stopped manually.
INFO:kafka.coordinator:Stopping heartbeat thread
INFO:kafka.coordinator:Leaving consumer group (my-group).
INFO:kafka.conn:<BrokerConnection node_id=2 host=ec2-54-224-168-37.compute-1.amazonaws.com:9094 <connected> [IPv4 ('54.224.168.37', 9094)]>: Closing connection. 
ERROR:kafka.consumer.fetcher:Fetch to node 2 failed: Cancelled: <BrokerConnection node_id=2 host=ec2-54-224-168-37.compute-1.amazonaws.com:9094 <connected> [IPv4 ('54.224.168.37', 9094)]>
INFO:kafka.conn:<BrokerConnection node_id=coordinator-2 host=ec2-54-224-168-37.compute-1.amazonaws.com:9094 <connected> [IPv4 ('54.224.168.37', 9094)]>: Closing connection. 
INFO:__main__:Consumer closed.
